# eICU data - dataset construction Labevents

In [1]:
import pandas as pd

pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 300)

In [2]:
patients = pd.read_csv('../2.0/patient.csv')
lab = pd.read_csv('../2.0/lab.csv')
patients_lab = patients.merge(lab, how='inner', on='patientunitstayid')

In [3]:
patients_lab = patients_lab[patients_lab['labresultoffset'] >=0 ]

In [4]:
#27 Variables overlapping between eICU data and MIMIC III data
subset = ['albumin','ALT (SGPT)','anion gap','AST (SGOT)','Base Excess','bicarbonate','total bilirubin','Total CO2','chloride',
          'creatinine','glucose','lactate','MCH','MCV','magnesium','PT','PTT','phosphate','platelets x 1000','potassium',
          'RDW','RBC','sodium','24 h urine urea nitrogen',"WBC's in body fluid",'paCO2','pH']

patients_lab = patients_lab.loc[patients_lab['labname'].isin(subset)]

In [5]:
frequent_labs = patients_lab.groupby('labname')['patientunitstayid'].nunique().sort_values(ascending=False)

In [6]:
long_labs = patients_lab.groupby('labname').apply(lambda x: x.groupby('patientunitstayid').size().mean()).sort_values(ascending=False)

In [7]:
selected_labs = set(list(long_labs.index)) & set(list(frequent_labs.index))

In [8]:
# To have a look at what lab measurements you have selected
labs_selected_df = pd.concat([frequent_labs.loc[selected_labs], long_labs.loc[selected_labs]], axis=1).rename({'patientunitstayid': 'icu_stays', 0: 'avg_length'}, axis=1)
print(labs_selected_df.to_string())

patients_lab_selected = patients_lab[patients_lab['labname'].isin(selected_labs)]

                          icu_stays  avg_length
labname                                        
total bilirubin               99659    3.162996
Total CO2                     26273    5.279450
paCO2                         68243    4.712351
pH                            67645    4.717156
creatinine                   180325    5.876928
PT                            81511    3.525230
sodium                       180373    6.324899
Base Excess                   55187    4.631544
AST (SGOT)                   102100    3.144094
anion gap                    144031    5.925599
ALT (SGPT)                   101477    3.142584
albumin                      109111    3.528068
chloride                     179453    5.921701
glucose                      179557    6.044170
RDW                          163314    5.266438
bicarbonate                  170272    5.848613
potassium                    180476    6.836987
WBC's in body fluid            1341    1.215511
RBC                          175531    5

In [9]:

patients_lab_selected['lab_times'] = (patients_lab_selected['labresultoffset'])

In [10]:
lab_dict=dict(zip(list(selected_labs),range(len(selected_labs))))
patients_lab_selected['lab_code']=patients_lab_selected['labname'].map(lab_dict)

In [11]:
patients_lab_selected_short = patients_lab_selected[['patientunitstayid', 'lab_times', 'lab_code', 'labresult']].rename({'patientunitstayid': 'id'}, axis=1)


In [12]:
patients_lab_selected_short_gb = patients_lab_selected_short.groupby(['id', 'lab_times', 'lab_code'], as_index=False).mean()

In [13]:
patients_lab_selected_short_gb.to_csv("lab_processed.csv")